In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
import numpy as np
import pandas as pd
import requests as r

# GUI : Input researcher name or ORCID

In [3]:
researcher = widgets.Text(value="Enter Researcher's name or ORCID") #
display(researcher)

Text(value="Enter Researcher's name or ORCID")

In [25]:
get_open_data_scores_for_researcher(researcher.value)


Fetching datasets from Zenodo
get_records_page - page_nr: 0
total_nr_of_hits: 1190
total_nr_of_pages: 119
next:https://zenodo.org/api/records/?sort=bestmatch&q=creators.name%3ACharlotte+Weil&page=2&size=10
get_records_page - page_nr: 1
get_records_page - page_nr: 2
get_records_page - page_nr: 3
get_records_page - page_nr: 4
get_records_page - page_nr: 5
get_records_page - page_nr: 6
get_records_page - page_nr: 7
get_records_page - page_nr: 8
get_records_page - page_nr: 9


KeyboardInterrupt: 

# Underlying main functions 

In [36]:
from researcher_to_data import api_zenodo
from researcher_to_publications import scholar_api
from publication_to_data import webscrape_datasets_from_publication

def get_datasets_of_researcher(researcher_name_or_orcid):
    """ Fetches Publications and data repositories, 
        Returns list of dataset (IN WHICHEVER FORMAT IVAN NEEDS!)"""
    
    datasets = []
    publications = []
    
    if researcher_name_or_orcid[0].isdigit():
        orcid = researcher_name_or_orcid
    
        print("Fetching datasets from Zenodo")
        datasets += api_zenodo.get_data_for_orcid(orcid)

        #print("Fetching datasets from Figshare")
        # 

        #print("Fetching datasets from DataVerse")
        #
        
        print("Fetching Publications DBs - not yet implemented for ORCID")
    
    else:
        name = researcher_name_or_orcid
        
        print("Fetching datasets from Zenodo")
        datasets += api_zenodo.get_data_for_name(name)
        
        #print("Fetching datasets from Figshare")
        #

        #print("Fetching datasets from DataVerse")
        #
        
        print("Fetching publications from Google Scholar")
        publications += scholar_api.main_scholar.get_publications_from_researcher(name)
        
        # print("Fetching publications from Scopus")
        # publications += ...
        
        # print("Fetching publications from WoS")
        # publications += ...
        
        print("Fetching datasets from identified publications")
        for pub in publications:
            ## Doesn't work.
            ## datasets += webscrape_datasets_from_publication.get_datasets_from_publication(pub)
            pass
        
        ###
        ### Remove any duplicated in datasets !!
        ### TO DO 
        ###
        
    return datasets
        

In [16]:
from data_to_metrics import data_to_metrics 

def get_open_data_scores_for_researcher(researcher_name_or_orcid,
                                        print_nb=True):
    """Returns Open Data Score for a reseacher"""
    
    datasets = get_datasets_of_researcher(researcher_name_or_orcid)
    
    scores = data_to_metrics.datasets_to_scores(datasets)
    
    if print_nb:
        print('Researcher: '+researcher_name_or_orcid)
        print_scores_nicely(scores)
    
def print_scores_nicely():
    ## To do Charlie for demo!!
    pass
    

In [37]:
d = api_zenodo.get_data_for_name(researcher.value)

get_records_page - page_nr: 0
total_nr_of_hits: 1190
total_nr_of_pages: 119
next:https://zenodo.org/api/records/?sort=bestmatch&q=creators.name%3ACharlotte+Weil&page=2&size=10
get_records_page - page_nr: 1
get_records_page - page_nr: 2
get_records_page - page_nr: 3
get_records_page - page_nr: 4
get_records_page - page_nr: 5
get_records_page - page_nr: 6
get_records_page - page_nr: 7
get_records_page - page_nr: 8


KeyboardInterrupt: 

In [32]:
d

In [12]:
api_zenodo.get_data_for_name?

In [33]:
d